# INHERITANCE & MULTIPLE DISPATCH


In [1]:
abstract type Employee end

struct Developer <: Employee    
    first::String
    last::String
    pay::Int64
    pro_lang::String
    raise_amount::Float64
    
    function Developer(first::String, last::String, pay::Int64, pro_lang::String, raise_amount = 1.1)
        new(first, last, pay, pro_lang, raise_amount)
    end
end


struct Coder <: Employee  
    first::String
    last::String
    pay::Int64
    raise_amount::Float64
    
    function Coder(first::String, last::String, pay::Int64, raise_amount = 1.04)
        new(first, last, pay, raise_amount)
    end
end


function fullname(a::Employee)
    return string(a.first," ", a.last)
end

function email(a::Employee)
    return string(a.first , "." , a.last , "@company.com")
end

function address()

LoadError: [91msyntax: incomplete: premature end of input[39m

In [2]:
dev1 = Developer("Corley", "Sharfer", 50000, "Julia")
dev2 = Developer("Test", "User", 6000, "Java")

@show fullname(dev1)
@show dev1.pro_lang
@show email(dev1)


coder1 = Coder("Corley", "Sharfer", 50000)
coder2 = Coder("Test", "User", 6000)

@show fullname(coder1) #the Coder class inherits the functions from its supper class
@show email(coder1)

fullname(dev1) = "Corley Sharfer"
dev1.pro_lang = "Julia"
email(dev1) = "Corley.Sharfer@company.com"
fullname(coder1) = "Corley Sharfer"
email(coder1) = "Corley.Sharfer@company.com"


"Corley.Sharfer@company.com"

In [3]:
abstract type CamClay end

In [4]:
type ModifiedCamClay <: CamClay
    params::Dict{String, Float64}
end

In [5]:
model = ModifiedCamClay(Dict("lambda" => 1.0, "kappa" => 0.2))

ModifiedCamClay(Dict("kappa"=>0.2,"lambda"=>1.0))

In [6]:
model.params

Dict{String,Float64} with 2 entries:
  "kappa"  => 0.2
  "lambda" => 1.0

In [7]:
type OriginalCamClay <: CamClay
    params::Dict{String, Float64}
end

In [8]:
function De(model::CamClay)
   return ones(6,6) 
end

De (generic function with 1 method)

In [9]:
De(model)

6×6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0

In [10]:
function NCL(model::CamClay, p::Real)
   p_ref = model.params["reference mean stress"]
   e_ref = model.params["reference void ratio"]
   λ = model.params["lambda"]
   return e_ref - λ*log(p/p_ref)
end

NCL (generic function with 1 method)

In [11]:
model = ModifiedCamClay(Dict(
        "lambda" => 1.0,
        "reference mean stress" => 98,
        "reference void ratio" => 0.8,))

ModifiedCamClay(Dict("reference mean stress"=>98.0,"lambda"=>1.0,"reference void ratio"=>0.8))

In [12]:
NCL(model, 200)

0.08665011212253526

In [13]:
function Base.getindex(model::CamClay, name::String)
    params = model.params
    return getindex(params, name)
end

In [14]:
function Base.setindex!(model::CamClay, name::String)
    params = model.params
    return setindex!(params, val, name)
end

In [15]:
model

ModifiedCamClay(Dict("reference mean stress"=>98.0,"lambda"=>1.0,"reference void ratio"=>0.8))

In [16]:
model.params["lambda"]

1.0

In [17]:
model["lambda"]

1.0

# UNION TYPE

In [18]:
type VariationalCamClay <: CamClay
    params::Dict{String, Float64}
end

In [19]:
function yieldfunction(model::Union{ModifiedCamClay, VariationalCamClay}, p::Float64, q::Float64)
   α = haskey(model.params, "alpha") ? model["alpha"] : 2
   return p + q + α
end

yieldfunction (generic function with 1 method)

In [20]:
yieldfunction(model, 10.0, 15.0)

27.0

In [21]:
model2 = VariationalCamClay(Dict(
              "lambda" => 1.0,
              "kappa" => 0.2,
              "alpha" => 3))


VariationalCamClay(Dict("alpha"=>3.0,"kappa"=>0.2,"lambda"=>1.0))

In [22]:
yieldfunction(model2, 10.0, 15.0)

28.0

# ANOTHER EXAMPLE OF UNION TYPE

In [23]:
type Point
    x::Float64
    y::Float64
end

type Vector2D
    x::Float64
    y::Float64
end

In [24]:
p = Point(2, 5)

Point(2.0, 5.0)

In [25]:
v = Vector2D(2,5)

Vector2D(2.0, 5.0)

Suppose we want to define the sum for these type as a point that has cooridinates as the sum of the corresponding coordinates

In [26]:
+(p, v)

LoadError: [91mMethodError: no method matching +(::Point, ::Vector2D)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

In [27]:
import Base.+
+(p::Point, v::Vector2D) = Point(p.x + v.x, p.y + v.y)

+ (generic function with 181 methods)

In [28]:
+(p, v)

Point(4.0, 10.0)

In [29]:
import Base.+

+(p::Point, v::Vector2D) = Point(p.x + v.x, p.y + v.y)
+(p::Point, v::Point)    = Point(p.x + v.x, p.y + v.y)
+(p::Vector, v::Point)   = Point(p.x + v.x, p.y + v.y)
+(p::Vector, v::Vector)  = Point(p.x + v.x, p.y + v.y)

+ (generic function with 184 methods)

Instead of writing 4 function as above, we can use UNION TYPE

In [30]:
VecOrPoint = Union{Vector2D, Point}

Union{Point, Vector2D}

In [31]:
+(p::VecOrPoint, v::VecOrPoint) = Point(p.x + v.x, p.y + v.y)

+ (generic function with 185 methods)

# PARAMETRIC TYPES AND METHODS

In [4]:
type Point{T}
    x::T
    y::T
end

In [5]:
x = Point(3, 5)

Point{Int64}(3, 5)

In [6]:
y = Point(3.1, 4.5)

Point{Float64}(3.1, 4.5)

In [7]:
typeof(x)

Point{Int64}

In [8]:
typeof(y)

Point{Float64}